<a href="https://colab.research.google.com/github/DmitriyValetov/nlp_course_project/blob/master/inferences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import numpy as np
from numpy.random import choice, randint

np.random.seed(0)
pad, unk, sos, eos = 0, 1, 2, 3
emb_size = 5  # 0, 1, 2, 3, 4
batch_size = 3
x1_lens = randint(1, 4, batch_size)
print(x1_lens)
max_len = np.max(x1_lens)
print(max_len)
x1_base = randint(1, 5, (batch_size, 4))
bx1 = np.array([[2] + list(choice([1, 4], x)) + [3] + [0] * (max_len - x) for x in x1_lens])
print(bx1)

def predict(bx1, bx2, emb_size=5):
  np.random.seed(0)
  batch_size, len_x2 = bx2.shape
  a = np.random.pareto(2, (len_x2, batch_size, emb_size))  # activations
  a = a.swapaxes(0, 1)  # for consitency
  s = np.exp(a)/np.sum(np.exp(a), axis=2, keepdims=True)  # softmax
  by2p = np.log(s)  # log softmax
  return by2p

import torch
from torch.distributions.pareto import Pareto
import torch.nn.functional as F

def torch_predict(bx1, bx2, emb_size=5):
  torch.manual_seed(0)
  batch_size, len_x2 = bx2.size()
  p = Pareto(torch.tensor([2.0]), torch.tensor([1.0]))  # Pareto
  a = p.sample((len_x2, batch_size, emb_size)).squeeze(-1)  # activations
  a = a.permute(1, 0, 2)
  by2p = F.log_softmax(a, 2)  # log softmax predictions
  return by2p

bx1 = torch.tensor(bx1)
bx2 = torch.full((batch_size, 1), 2)
torch_predict(bx1, bx2)

[1 2 1]
2
[[2 1 3 0]
 [2 1 1 3]
 [2 4 3 0]]


tensor([[[ 0.0000e+00, -5.9940e+01, -6.3085e+01, -4.1549e+01, -6.1150e+01]],

        [[-1.1626e-02, -7.1529e+00, -6.5056e+00, -4.8106e+00, -6.7779e+00]],

        [[-2.1067e+00, -3.3326e+00, -2.6601e+00, -4.7244e-01, -1.9022e+00]]])

In [0]:
def greedy(bx1, sos=2, eos=3, max_len=10):
  bx2 = np.full((batch_size, 1), sos)  # batch with <sos> token
  # stop when predictions len > max_len or all have <eos> token
  while 1 + bx2.shape[1] < max_len and not np.all(np.any(bx2 == eos, axis=1)):
    # print(bx2)
    by2p = predict(bx1, bx2)
    lp = by2p[:,-1,:]  # last prediction
    # print(by2)
    # print(lp)
    next_bx2 = np.argmax(lp, axis=1).reshape((lp.shape[0], 1))
    # print(next_bx2)
    bx2 = np.concatenate((bx2, next_bx2), axis=1)
    # print(bx2)
    # print(bx2 == eos)
    # print(np.all(np.any(bx2 == eos, axis=1)))
  return by2p

def greedy_many_to_one(bx1, sos=2, eos=3, max_len=10, reduction=np.sum):
  bx2 = np.full((bx1.shape[0], 1), sos)  # batch with <sos> token
  # stop when predictions len > max_len or all have <eos> token
  while 1 + bx2.shape[1] < max_len and not np.all(np.any(bx2 == eos, axis=1)):
    print(bx2)
    by2p = predict(bx1, bx2)
    lp = by2p[:,-1,:]  # last prediction
    print(lp)
    lp_red = reduction(lp, axis=0)
    print(lp_red)
    # print(by2)
    # print(lp)
    # next_bx2 = np.argmax(lp, axis=1).reshape((lp.shape[0], 1))
    next_x2 = np.argmax(lp_red, axis=0)
    # print(next_x2)
    next_bx2 = np.full((lp.shape[0], 1), next_x2)
    # print(next_bx2)
    bx2 = np.concatenate((bx2, next_bx2), axis=1)
    # print(bx2)
    # print(bx2 == eos)
    # print(np.all(np.any(bx2 == eos, axis=1)))
  return by2p

def forced(bx1, bx2):
  by2p = predict(bx1, bx2)
  return by2p

def many_to_one(by2p, reduction=np.sum):
  y2p = np.sum(by2p[:,:-1,:], axis=0)
  y2 = np.argmax(y2p, axis=1)
  return y2

# by2p = greedy_many_to_one(bx1, reduction=np.sum)
# print(by2p)
# by2p = greedy_many_to_one(bx1, reduction=np.max)
# print(by2p)
# by2p = greedy_many_to_one(bx1, reduction=np.mean)
# print(by2p)
# y2 = np.argmax(np.sum(by2p[:,:-1,:], axis=0), axis=1)
# print(y2)
# print(many_to_one_sum(by2p))
# by2p = greedy(bx1)
# print(by2p)
# by2 = np.argmax(by2p[:,:-1,:], axis=2)
# print(by2)
# print(many_to_one(by2p, np.sum))
# print(many_to_one(by2p, np.max))
# print(many_to_one(by2p, np.mean))
# forced
# bx2 = np.concatenate((np.full((batch_size, 1), 2), by2), axis=1)
# print(bx2)
# by2p = forced(bx1, bx2)
# # print(bpy2)
# by2 = np.argmax(by2p[:,:-1,:], axis=2)
# print(by2)

In [267]:
# TODO do many_to_one search with batch_reduction
# TODO try Viterbi algorithm for best beam searching (like beam search but in reversed direction)
# TODO maybe try spectral beams? (With multiple reductions + Viterbi)
# bx1 - encoder batch input
# bx2 - decoder batch input
# by2p - encoder batch predictions

def beam(bx1, sos=2, eos=3, max_len=6,
         beam_width=2, beam_depth=2, depth_reduction=np.sum):
  batch_size = bx1.shape[0]
  bx2 = np.full((batch_size, 1), sos)  # base batch with <sos> token
  cur_len = bx2.shape[1] - 1  # without <sos>
  # stop when cur_len > max_len or all predictions have <eos> token
  while cur_len < max_len and not np.all(np.any(bx2 == eos, axis=1)):
    bx2t = bx2.copy()  # temporal batch bx2
    for i in range(beam_depth):
      by2p = predict(bx1, bx2t)  # all predictions
      lp = by2p[:,-1,:]  # last prediction
      next_bx2 = np.argsort(-lp, axis=1)[:,:beam_width]  # top "beam_width" last predictions
      bx2t = np.repeat(bx2t, beam_width, axis=0)  # multiply batch_size by beam_width
      next_bx2 = next_bx2.reshape(batch_size*beam_width**(i+1), 1)  # to new batch_size
      bx2t = np.concatenate((bx2t, next_bx2), axis=1)  # update batch
    # Prediction with temporal batch
    by2p = predict(bx1, bx2t)  # all predictions by last temporal batch
    by2p = by2p[:,cur_len:-1]  # [base part:-last]  # remove old predictions
    bx2t = bx2t[:,cur_len+1:]  # [base part + <sos>:]  # remove old labels
    # Best beams searching
    by2p = np.take_along_axis(by2p, bx2t[:,np.newaxis], axis=2)  # beams transitions predictions
    by2p = by2p.reshape(batch_size, beam_width**beam_depth, -1)  # reshape to base batch shape (like)
    by2p_red = depth_reduction(by2p, axis=2)  # beams reduction, i.e. giving them scores
    best_beams = np.argmax(by2p_red, axis=1)  # get indices of the best beams
    # Base batch updating
    bx2t = bx2t.reshape(batch_size, beam_width**beam_depth, -1)  # reshape to base batch shape (like)
    bx2t = np.take_along_axis(bx2t, best_beams[:, np.newaxis, np.newaxis], axis=1)  # get best beams
    bx2t = bx2t.reshape(batch_size, -1)  # to new labels to base batch_size
    bx2 = np.concatenate((bx2, bx2t), axis=1)  # update base batch
    cur_len = bx2.shape[1] - 1  # without <sos>
  if bx2.shape[1] - 1 > max_len:  # cut to max_len
    bx2 = bx2[:max_len + 1]  # <sos> included
  return bx2

bx2 = beam(bx1, depth_reduction=np.sum)
print(bx2)
bx2 = beam(bx1, depth_reduction=np.mean)
print(bx2)

[[2 2 3 0 0 3 1]
 [2 2 3 1 4 1 0]
 [2 0 1 2 4 4 1]
 [2 2 3 3 4 3 2]
 [2 1 3 2 3 2 2]]
[[2 2 3 0 0 3 1]
 [2 2 3 1 4 1 0]
 [2 0 1 2 4 4 1]
 [2 2 3 3 4 3 2]
 [2 1 3 2 3 2 2]]


In [268]:
# TODO do many_to_one search with batch_reduction
# TODO try Viterbi algorithm for best beam searching (like beam search but in reversed direction)
# TODO maybe try spectral beams? (With multiple reductions + Viterbi)
# bx1 - encoder batch input
# bx2 - decoder batch input
# by2p - encoder batch predictions

def torch_beam(bx1, sos=2, eos=3, max_len=10,
               beam_width=2, beam_depth=2, depth_reduction=torch.sum):
  batch_size = bx1.size(0)  # input batch_size
  bx2 = torch.full((batch_size, 2), sos, dtype=bx1.dtype)  # batch with <sos>
  # stop when len decoder output > max_len or all decoder outputs have <eos> token
  while bx2.size(1) - 1 < max_len and not torch.all(torch.any(bx2 == eos, axis=1)):
    beam_scores = torch.empty((batch_size, 0))  # scores for each beam
    for i in range(beam_depth):
      new_batch_size = batch_size*beam_width**(i+1)
      by2p = torch_predict(bx1, bx2)  # predict
      next_by2p, next_bx2 = torch.topk(by2p[:,-1,:], beam_width)  # beams to top k last predictions
      next_bx2 = next_bx2.view(new_batch_size, 1)  # new beams
      next_by2p = next_by2p.view(new_batch_size, 1)  # new scores
      beam_scores = torch.repeat_interleave(beam_scores, beam_width, dim=0)  # increase batch for new scores
      bx2 = torch.repeat_interleave(bx2, beam_width, dim=0)  # increase batch for new beams
      bx2 = torch.cat((bx2, next_bx2), 1)  # add beams
      beam_scores = torch.cat((beam_scores, next_by2p), 1)  # add beams scores
    beam_scores = depth_reduction(beam_scores, axis=1) # cumulative beams scores
    beam_scores = beam_scores.view(batch_size, -1)  # split scores into batches
    best_beams = torch.argmax(beam_scores, axis=1, keepdim=True)  # best beams
    bx2 = bx2.view(batch_size, beam_width**beam_depth, -1)  # split beams into batches
    # XXX its fucking magic... (return to input batch_size)
    best_beams = best_beams.unsqueeze(2).expand(best_beams.size(0), best_beams.size(1), bx2.size(2))
    bx2 = torch.gather(bx2, 1, best_beams)
    bx2 = bx2.view(batch_size, -1)
  return bx2

bx1 = torch.full((5, 1), 0, dtype=torch.long)
bx2 = torch_beam(bx1, max_len=10,
                 beam_width=2, beam_depth=2, depth_reduction=torch.sum)
print(bx2)
bx2 = torch_beam(bx1, depth_reduction=torch.mean)
print(bx2)

tensor([[2, 2, 3, 1, 3, 3, 0, 0],
        [2, 2, 0, 0, 1, 2, 3, 1],
        [2, 2, 0, 3, 3, 1, 2, 4],
        [2, 2, 2, 3, 2, 4, 1, 0],
        [2, 2, 3, 1, 3, 1, 3, 4]])
tensor([[2, 2, 3, 1, 3, 3, 0, 0],
        [2, 2, 0, 0, 1, 2, 3, 1],
        [2, 2, 0, 3, 3, 1, 2, 4],
        [2, 2, 2, 3, 2, 4, 1, 0],
        [2, 2, 3, 1, 3, 1, 3, 4]])


In [0]:
import torch
bt = bx1[:,1:]  # without <sos>
print(bt)
print(bt[bt != 0])
bp = by2[:,:-1,:]  # without last prediction
print(bp)
print(bt != 0)
print(bp[bt != 0])

bp, bt = torch.tensor(bp), torch.tensor(bt)
loss = torch.nn.NLLLoss()
print(loss(bp[bt != 0], bt[bt != 0]))

loss = torch.nn.NLLLoss(ignore_index=0)
print(loss(torch.flatten(bp, 0, 1), torch.flatten(bt, 0)))

loss = 0
cnt = 0
for i, t in enumerate(bt):
  for j, l in enumerate(t):
    print(l)
    print(bp[i,j,l])
    loss += bp[i,j,l].item()
    cnt += 1
print(loss, loss/cnt)